# Is it in Stock?
Al vele jaren gaat Klaas naar de supermarkt bij hem om de hoek. Hij doet er niet alleen boodschappen voor zichzelf, maar ook voor zijn mindervalide buurvrouw. Elke dinsdag krijgt hij een boodschappenlijstje mee en brengt hij de boodschappen weer netjes bij haar thuis af.
Klaas heeft echter recent een extra baan gekregen en wil dit proces optimaliseren om zo meer tijd te besparen. Aan de hand van de volgende opdrachten ga jij hem helpen zo efficient mogelijk de boodschappen te doen.

### Opdracht 1

Adrie heeft het boodschappenlijstje voor hem klaargelegd. Zijn alle producten op het lijstje ook te vinden in de supermarkt? Het assortiment is hier te vinden.

### Opdracht 2
Klaas krijgt altijd 15 euro mee om de boodschappen mee te doen. Heeft Klaas heir genoeg aan? De prijzen zijn weer te vinden in het assortiment.

### Opdracht 3
De buurvrouw is jarig en wilt graag een taart bakken. Aangezien ze niet zo van het weggooien is wilt ze altijd goed kijken naar de hoeveelheid ingredienten en niet teveel kopen. Hoeveel blijft er over van de ingredienten voor het taartrecept, als de boodschappen zo gedaan worden met de minste verspilling?

### Bonus
Binnenin de supermarkt is de plek van de producten bekend. Met behulp van Dijkstra's algoritme kan je kijken wat de kortste route in de supermarkt is. Zo kan je de looproute optimaliseren en dus weer sneller thuis zijn.

Hieronder is een schematische weergave van de supermarkt toegevoegd. Bereken de korste route aan de hand van het boodschappenlijstje van de eerste opdracht. Uiteraard moet je langs elk schap van het product op je lijstje.

## Uitwerking

Om te beginnen lezen we alle bestanden in.

In [1]:
import pandas as pd
import json
import re

with open("lijstje.json") as f:
    lijst_json = json.load(f)
    
df_lijst = pd.json_normalize(lijst_json["lijstje"])

with open("assortiment.json") as f:
    assor_json = json.load(f)

with open("recept.json") as f:
    recept_json = json.load(f)

Om te controleren of alle producten van het boodschappenlijstje aanwezig zijn, moeten we checken of deze voorkomen in de assortiments lijst.

De uitdaging hierin zit in dat sommige namen niet exact overeen komen, met name paprika chips. Er zijn meerdere producten met het woord paprika erin en er is ook een andere smaak chips.

Om dit op te lossen heb ik gekozen voor een regular expression. Hierin kun je aan de hand van match groepen, de volgorde waarin deze voorkomen omdraaien en tevens onderscheid maken tussen matches die uit 1 woord bestaan, of uit meerdere.

Nadat naar elk product is gezocht, wordt het boodschappenlijstje aangevuld met een kolom waarin wordt aangegeven of een product aanwezig is of niet.

Ter voorbereiding van de volgende opdrachten hernoemen we de producten in het assortiment. Het onderscheid tussen een klein of grote verpakking kunnen we maken aan de hand van andere data (inhoud).

In [2]:
boodschappenlijst = lijst_json["lijstje"]
df_lijst = pd.json_normalize(boodschappenlijst)

def check_availability(df):
    productnaam = df["product"].values.tolist()
    boodschappen_gevonden = {}
    
    for product in productnaam:
        m = re.match("(\w*) ?(\w*)", product)

        for artikel in assor_json["producten"]:
            if m[2]:
                m2 = re.match(f"{m[1]}.*{m[2]}|{m[2]}.*{m[1]}", artikel["product"])
                if m2:
                    print(f"gevonden artikelen voor {m[0]}:")
                    print(artikel["product"])
                    boodschappen_gevonden[product] = True
                    artikel["product"] = m[0]
            elif m[1]:
                    m3 = re.match(m[1], artikel["product"])
                    if m3:
                        print(f"gevonden artikelen voor {m[0]}:")
                        print(artikel["product"])
                        boodschappen_gevonden[product] = True
                        artikel["product"] = m[0]

    df["gevonden"] = df["product"]
    df["gevonden"] = df["gevonden"].map(boodschappen_gevonden)

check_availability(df_lijst)
df_lijst

gevonden artikelen voor melk:
melk (literpak)
gevonden artikelen voor melk:
melk (groot)
gevonden artikelen voor gildekorn:
gildekorn
gevonden artikelen voor cassis:
cassis
gevonden artikelen voor milde kwark:
milde kwark
gevonden artikelen voor mars:
mars
gevonden artikelen voor paprika chips:
chips (paprika)
gevonden artikelen voor eieren:
eieren (6-pak)
gevonden artikelen voor eieren:
eieren (dozijn)


,product,hoeveelheid,eenheid,gevonden
0,melk,2.4,liter,True
1,gildekorn,2.0,stuks,True
2,cassis,1.0,liter,True
3,milde kwark,500.0,gram,True
4,mars,1.0,multipack,True
5,paprika chips,1.0,zak,True
6,eieren,8.0,stuks,True


Om te kijken of 15 euro genoeg is om alles op de boodschappenlijst te kopen, moeten we een berekening maken met de aantallen op het lijstje en wat er in het assortiment staat.

Er is niet aangegeven wat we moeten doen als de gevraagde hoeveelheid niet overeen komt met de verkoop hoeveelheid (bijvoorbeeld Eieren), dus in dit geval kiezen we de goedkoopste optie die aan de gevraagde hoeveelheid voldoet.

We mergen het dataframe van het assortiment met die van de boodschappen lijst. Zo hebben we 1 dataframe waarin alle benodigde informatie staat.

In [3]:
df_assor = pd.json_normalize(assor_json["producten"])

df_lijst.rename(columns={"eenheid":"boodschappen_eenheid"}, inplace=True)
df_lijst2 = df_lijst.merge(df_assor, how="inner")

df_lijst2["ppu"] = df_lijst2["prijs"] / df_lijst2["gewicht"]

df_lijst2

,product,hoeveelheid,boodschappen_eenheid,gevonden,schap,gewicht,eenheid,prijs,ppu
0,melk,2.4,liter,True,koelschap,1.0,liter,1.19,1.190000
1,melk,2.4,liter,True,koelschap,2.4,liter,2.24,0.933333
2,gildekorn,2.0,stuks,True,brood,1.0,brood,1.87,1.870000
3,cassis,1.0,liter,True,dranken,1.0,liter,1.37,1.370000
4,milde kwark,500.0,gram,True,koelschap,500.0,gram,1.19,0.002380
5,mars,1.0,multipack,True,snacks,1.0,multipack,2.90,2.900000
6,paprika chips,1.0,zak,True,snacks,200.0,gram,1.14,0.005700
7,eieren,8.0,stuks,True,bakproducten,6.0,stuks,2.14,0.356667
8,eieren,8.0,stuks,True,bakproducten,12.0,stuks,3.68,0.306667


Er staat nu 2 verschillende hoeveelheden in voor de melk en voor de eieren. We hebben echter berekend welke pro rata het goedkoopst is.

Als we het dataframe sorteren op `product` en vervolgens op de `prijs per eenheid`, dan weten we zeker dat de goedkoopste optie bovenaan staan.

Daardoor kunnen we de `drop_duplicates` methode met het argument `keep=first` gebruiken om er 1 over te houden. Met dit argument blijft van de dubbele waardes alleen degene die als eerst voorkomt staan.

In [4]:
df_lijst2.sort_values(by=["product", "ppu"], inplace=True)
df_lijst2.drop_duplicates(subset="product", inplace=True)

df_lijst2

,product,hoeveelheid,boodschappen_eenheid,gevonden,schap,gewicht,eenheid,prijs,ppu
3,cassis,1.0,liter,True,dranken,1.0,liter,1.37,1.370000
8,eieren,8.0,stuks,True,bakproducten,12.0,stuks,3.68,0.306667
2,gildekorn,2.0,stuks,True,brood,1.0,brood,1.87,1.870000
5,mars,1.0,multipack,True,snacks,1.0,multipack,2.90,2.900000
1,melk,2.4,liter,True,koelschap,2.4,liter,2.24,0.933333
4,milde kwark,500.0,gram,True,koelschap,500.0,gram,1.19,0.002380
6,paprika chips,1.0,zak,True,snacks,200.0,gram,1.14,0.005700


Het enige probleem dat nu overblijft is dat de chips als 1 zak is opgegeven, terwijl deze per gram in het assortiment staan.

Omdat het maar 1 product is wat dit 'probleem' heeft, is het nu het efficientst om dit handmatig aan te passen.

In [5]:
df_lijst2.at[6, "gewicht"] = 1.0
df_lijst2.at[6, "eenheid"] = "zak"

Daarna berekenen we het benodigde aantal door de bestelde hoeveelheid te delen door de verkoop aantallen.

De eieren worden verkocht per 12, maar er staan er 8 op het lijstje. Hierdoor is de uitkomst van bovenstaande deling 0.6667
Via `numpy` kunnen we een decimaal getal naar boven afronden.

Dit aantal vermenigvuldigen we met de prijs om het subtotaal wat we per product uitgeven te krijgen.

Daarna printen we de som van het subtotaal.

In [6]:
import numpy as np

# Bereken benodigde aantallen
df_lijst2["sub_aantal"] = df_lijst2["hoeveelheid"] / df_lijst2["gewicht"]
df_lijst2["sub_aantal"] = df_lijst2["sub_aantal"].apply(np.ceil)

df_lijst2["subtotaal"] = df_lijst2["sub_aantal"] * df_lijst2["prijs"]

totaal_boodschappen = df_lijst2["subtotaal"].sum()

if df_lijst2["subtotaal"].sum() <= 15:
    print(f"Het totaal is {totaal_boodschappen}euro. We kunnen alles op het boodschappenlijstje kopen voor 15 euro!")
else:
    print(f"Helaas! 15 euro is niet genoeg om alle boodschappen van te doen. Het totaal is {totaal_boodschappen} euro.")

df_lijst2

Helaas! 15 euro is niet genoeg om alle boodschappen van te doen. Het totaal is 16.26 euro.


,product,hoeveelheid,boodschappen_eenheid,gevonden,schap,gewicht,eenheid,prijs,ppu,sub_aantal,subtotaal
3,cassis,1.0,liter,True,dranken,1.0,liter,1.37,1.370000,1.0,1.37
8,eieren,8.0,stuks,True,bakproducten,12.0,stuks,3.68,0.306667,1.0,3.68
2,gildekorn,2.0,stuks,True,brood,1.0,brood,1.87,1.870000,2.0,3.74
5,mars,1.0,multipack,True,snacks,1.0,multipack,2.90,2.900000,1.0,2.90
1,melk,2.4,liter,True,koelschap,2.4,liter,2.24,0.933333,1.0,2.24
4,milde kwark,500.0,gram,True,koelschap,500.0,gram,1.19,0.002380,1.0,1.19
6,paprika chips,1.0,zak,True,snacks,1.0,zak,1.14,0.005700,1.0,1.14


De buurvrouw wil een taart bakken. We hebben het complete recept gekregen, waarbij de ingredienten en de hoeveelheid op dezelfde regel staan.

Deze willen we opsplitsen om een boodschappenlijst zoals we hiervoor hadden te krijgen.

In [7]:
recept_ingredienten = recept_json["recept"]["ingredienten"]
print(recept_ingredienten)
df_taart = pd.DataFrame(columns=["product", "hoeveelheid", "recept_eenheid"])

for regel in recept_ingredienten:
    ingredienten_dict = {}
    m = re.match("(\d+)(\w*) (\w*)", regel)
    ingredienten_dict["product"] = m[3]
    ingredienten_dict["hoeveelheid"] = m[1]
    if m[2]:
        ingredienten_dict["recept_eenheid"] = m[2]
    else:
        ingredienten_dict["recept_eenheid"] = "stuks"
    df_ingredient = pd.DataFrame([ingredienten_dict])
    df_taart = pd.concat([df_taart, df_ingredient], ignore_index=True)
    
df_taart

['80g boter', '4 eieren', '250ml melk', '100g bloem', '4 appels']


,product,hoeveelheid,recept_eenheid
0,boter,80,g
1,eieren,4,stuks
2,melk,250,ml
3,bloem,100,g
4,appels,4,stuks


Nu hebben we de boodschappenlijst voor de taart. We stoppen dit dataframe in de functie die we eerder hebben gemaakt om te kijken of elk product aanwezig is.

Er was een probleem met de Appels, deze staan in het meervoud in het recept en als enkelvoud in het assortiment.
We kunnen via string slicing de regex in de functie aanpassen om de laatste letter te excluden, maar omdat het om 1 regel gaat, wordt dat omslachtiger dan die ene regel handmatig aanpassen.

Verder staat er niet beschreven of de boter gezouten of ongezouten moet zijn, maar meestal wordt gezouten boter gebruikt voor een appeltaart [(bron)](https://www.sophiefoodblog.nl/post/2019/05/18/appeltaart).

Daarna mergen we deze producten weer met het assortiment dataframe.

In [16]:
df_taart.at[4, "product"] = "appel"
df_taart.loc[df_taart["product"] == "boter", "product"] = "boter gezouten"

check_availability(df_taart)

df_taart2 = df_taart.merge(df_assor, how="inner")
df_taart2

gevonden artikelen voor boter gezouten:
boter gezouten
gevonden artikelen voor boter gezouten:
boter gezouten
gevonden artikelen voor eieren:
eieren
gevonden artikelen voor eieren:
eieren
gevonden artikelen voor melk:
melk
gevonden artikelen voor melk:
melk
gevonden artikelen voor bloem:
bloem
gevonden artikelen voor appel:
appel


,product,hoeveelheid,recept_eenheid,gevonden,schap,gewicht,eenheid,prijs
0,boter gezouten,80,g,True,koelschap,200.0,gram,1.49
1,eieren,4,stuks,True,bakproducten,6.0,stuks,2.14
2,eieren,4,stuks,True,bakproducten,12.0,stuks,3.68
3,melk,250,ml,True,koelschap,1.0,liter,1.19
4,melk,250,ml,True,koelschap,2.4,liter,2.24
5,bloem,100,g,True,bakproducten,1.0,kg,0.67
6,appel,4,stuks,True,groente,1.0,kg,2.19


We zien dat de eenheden niet kloppen. De eenheden van het melk en bloem kunnen we omrekenen, maar de appels worden verkocht per kilo ipv per stuk.

Na wat zoekwerk is het [gemiddelde gewicht per appel](https://be.meridianfarmersmarket.org/6680-what-is-the-mass-of-one-apple.html): 150 gram

Hieronder converten we alles naar de juiste eenheden, waarbij het aantal appels per kilo naar boven wordt afgerond.

In [17]:
# convert appel kg naar stuks (1 appel = ~150g)
df_taart2.loc[df_taart2["product"] == "appel", "gewicht"] = round(df_taart2.loc[df_taart2["product"] == "appel", "gewicht"] * 1000 / 150)
df_taart2.loc[df_taart2["product"] == "appel", "eenheid"] = "stuks"

# convert Liter to ml
df_taart2.loc[df_taart2["eenheid"] == "liter", "gewicht"] = df_taart2.loc[df_taart2["eenheid"] == "liter", "gewicht"] * 1000
df_taart2.loc[df_taart2["eenheid"] == "liter", "eenheid"] = "ml"

# convert Kg to g
df_taart2.loc[df_taart2["eenheid"] == "kg", "gewicht"] = df_taart2.loc[df_taart2["eenheid"] == "kg", "gewicht"] * 1000
df_taart2.loc[df_taart2["eenheid"] == "kg", "eenheid"] = "g"

df_taart2

,product,hoeveelheid,recept_eenheid,gevonden,schap,gewicht,eenheid,prijs
0,boter gezouten,80,g,True,koelschap,200.0,gram,1.49
1,eieren,4,stuks,True,bakproducten,6.0,stuks,2.14
2,eieren,4,stuks,True,bakproducten,12.0,stuks,3.68
3,melk,250,ml,True,koelschap,1000.0,ml,1.19
4,melk,250,ml,True,koelschap,2400.0,ml,2.24
5,bloem,100,g,True,bakproducten,1000.0,g,0.67
6,appel,4,stuks,True,groente,7.0,stuks,2.19


Voor de eieren en de melk willen we nu de kleinste verpakking, dus dit keer sorteren we op de inhoudt.

Daarna kunnen we de hoeveelheden in het recept aftrekken van de verkoopt hoeveelheid.

In [18]:
df_taart2.sort_values(by=["product", "gewicht"], inplace=True)
df_taart2.drop_duplicates(subset="product", inplace=True)

df_taart2["hoeveelheid"] = df_taart2["hoeveelheid"].astype("float")
df_taart2["restant"] = df_taart2["gewicht"] - df_taart2["hoeveelheid"]

df_taart2["restant_eenheid"] = df_taart2["eenheid"]

df_taart2[["product", "restant", "restant_eenheid"]]

,product,restant,restant_eenheid
6,appel,3.0,stuks
5,bloem,900.0,g
0,boter gezouten,120.0,gram
1,eieren,2.0,stuks
3,melk,750.0,ml


## Deze week in de bonus: Dijkstra's Algoritme

Dijkstra's algoritme is een algoritme dat gebruikt wordt om de kortste route van punt A naar punt B te berekenen. Er wordt gebruikt gemaakt van een "graph" waarin verschillende punten zijn (genaamd "Nodes"). Tussen elke node zit een pad (genaamd "edge"). Deze hebben een bepaalde waarde, in ons geval "afstand" (maar dit zou ook iets anders kunnen zijn waarmee je de route wilt optimaliseren; snelste route, mooiste route, route met de minste straatklinkers :)).

Er wordt een startpunt aangegeven en het algoritme gaat van daaruit naar elk aangrenzende Node. Hierbij wordt de afstand van de edge daar naartoe opgeslagen. Als alle buur Nodes bezocht zijn, gaan het algoritme verder naar de volgende nodes, hierbij wordt de afstand van de nieuwe Edge opgeteld en worden de totale afstand van startpunt naar huidige Node opgeslagen. Als er via een andere node een kortere afstand wordt gevonden, overschrijft deze.

Uiteindelijk krijg je een soort database met voor elke Node met het kortste pad hoe je bij die node kunt komen. 

Enkele links die ik heb gebruikt om het algoritme beter te begrijpen:

[Dijkstra's Algoritme op Wikipedia](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) (alhoewel ik deze niet het meest duidelijk vond)

[Computerphile op Youtube](https://www.youtube.com/watch?v=GazC3A4OQTE) (Hier wordt fysiek met papiertjes uitgelegd hoe het werkt, beetje langdradig maar wel heel duidelijk)

[Deze blog](https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html) (Hele duidelijke uitleg met afbeeldingen en voorbeeld code)

Ik heb de code in de blog als voorbeeld gebruikt. Initieel werkte die niet, maar nadat wat debugging heb ik het werkend gekregen.

In [11]:
import copy

class Graph:
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        """Construct graph from all nodes"""
        graph = {}
        for node in nodes:
            graph[node] = {}
        
        graph.update(init_graph)
        
        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if graph[adjacent_node].get(node, False) == False:
                    graph[adjacent_node][node] = value
        
        return graph
    
    def get_nodes(self):
        """Returns the node of the graph"""
        return self.nodes
    
    def get_outgoing_edges(self, node):
        """Returns all neighbouring nodes"""
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections
    
    def value(self, node1, node2):
        """Returns the value of an edge between two nodes"""
        return self.graph[node1][node2]

def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = copy.deepcopy(graph.get_nodes())
    previous_nodes = {}
    shortest_path = {node: np.inf for node in unvisited_nodes}
    shortest_path[start_node] = 0
    
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes:
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
                
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                previous_nodes[neighbor] = current_min_node
        unvisited_nodes.remove(current_min_node)
        
    return previous_nodes, shortest_path

def print_result(previous_nodes, shortest_path, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]

    path.append(start_node)
    
    print(f"Het kortste pad is {shortest_path[target_node]} supermarkt-pad-lengte-eenheden lang")
    print(" -> ".join(reversed(path)))
    
    return path

We hebben het algoritme gedefinieert. Nu moeten we onze `graph` van de supermarkt maken waarin we alle schappen en hun onderlinge afstanden toevoegen.

<img src="supermarkt.png" width="500" height="250" alt="Supermarkt Layout">

In [12]:
alle_schappen = df_assor["schap"]
alle_schappen.drop_duplicates(inplace=True)
alle_schappen = alle_schappen.values.tolist()
alle_schappen = alle_schappen + ["ingang", "kassa", "uitgang"]

init_graph = {}

for node in alle_schappen:
    init_graph[node] = {}

init_graph["ingang"]["groente"] = 4
init_graph["ingang"]["brood"] = 6
init_graph["groente"]["ingang"] = 4
init_graph["groente"]["brood"] = 8
init_graph["groente"]["snacks"] = 6
init_graph["groente"]["koelschap"] = 5
init_graph["groente"]["non-food"] = 12
init_graph["brood"]["snacks"] = 9
init_graph["brood"]["ingang"] = 6
init_graph["snacks"]["non-food"] = 5
init_graph["snacks"]["koelschap"] = 11
init_graph["snacks"]["brood"] = 9
init_graph["snacks"]["groente"] = 6
init_graph["non-food"]["koelschap"] = 12
init_graph["non-food"]["bakproducten"] = 7
init_graph["non-food"]["diepvries"] = 3
init_graph["non-food"]["groente"] = 12
init_graph["non-food"]["snacks"] = 5
init_graph["koelschap"]["bakproducten"] = 5
init_graph["koelschap"]["diepvries"] = 8
init_graph["koelschap"]["groente"] = 5
init_graph["koelschap"]["snacks"] = 11
init_graph["koelschap"]["non-food"] = 12
init_graph["bakproducten"]["dranken"] = 6
init_graph["bakproducten"]["kassa"] = 5
init_graph["bakproducten"]["diepvries"] = 8
init_graph["bakproducten"]["non-food"] = 7
init_graph["bakproducten"]["koelschap"] = 5
init_graph["diepvries"]["kassa"] = 3
init_graph["diepvries"]["non-food"] = 3
init_graph["diepvries"]["koelschap"] = 8
init_graph["diepvries"]["bakproducten"] = 8
init_graph["dranken"]["kassa"] = 6
init_graph["dranken"]["bakproducten"] = 6
init_graph["kassa"]["uitgang"] = 5

graph = Graph(alle_schappen, init_graph)

previous_nodes, shortest_path = dijkstra_algorithm(graph=graph, start_node="ingang")
shortest_route = print_result(previous_nodes, shortest_path, start_node="ingang", target_node="uitgang")

Het kortste pad is 24 supermarkt-pad-lengte-eenheden lang
ingang -> groente -> koelschap -> bakproducten -> kassa -> uitgang


Dit is het kortste pad van de ingang van de supermarkt tot de uitgang. Nu gaan we kijken of we na het volgen van dit pad alle boodschappen in het mandje hebben

In [13]:
# Haal alle afdelingen op van het boodschappenlijstje
df_dijkstra = df_lijst2["schap"]
df_dijkstra.drop_duplicates(inplace=True)
boodschappenlijst_path = df_dijkstra.values.tolist()

# Haal alle afdelingen op van het taart recept
df_dijkstra2 = df_taart2["schap"]
df_dijkstra2.drop_duplicates(inplace=True)
taart_path = df_dijkstra2.values.tolist()

def check_path(path, shortest_route):
    counter = 0
    for schap in path:
        if schap in shortest_route:
            counter += 1
            
    if len(path) == counter:
        print("Alle benodigde afdelingen zijn bezocht!")
    else:
        print("Er missen een aantal afdelingen op deze route...")

print(f"Boodschappenlijst:")
check_path(boodschappenlijst_path, shortest_route)
print(f"Taartrecept:")
check_path(taart_path, shortest_route)

Boodschappenlijst:
Er missen een aantal afdelingen op deze route...
Taartrecept:
Alle benodigde afdelingen zijn bezocht!


Er missen wat boodschappen van de lijst, dus we moeten met een soort 'waypoints' gaan werken.

In de originele functie hebben we als argumenten het startpunt en eindpunt en we krijgen een lijst met afstanden vanaf het startpunt naar alle schappen terug. 

We kunnen dus steeds controleren welk schap van het boodschappenlijstje het dichtsbijzijnd is, deze als startpunt nemen en het algoritme opnieuw uitvoeren om het eerst volgende schap op het lijstje te krijgen.

Doordat we eenzelfde soort zoekopdracht meerdere keren uitvoeren, dacht ik aan de vorige Sudoku opdracht, waarin we een solver hebben gemaakt die via recursie de juiste oplossing vind.

We kunnen een nieuwe functie maken die het Dijkstra algoritme runt voor elk schap op de lijst, tot we alle schappen bezocht hebben, en dan de kortste route naar de uitgang.

In [20]:
# make deepcopy so this cell can be re-run
b_path = copy.deepcopy(boodschappenlijst_path)

def check_waypoints(waypoints, start_node, target_node):
    """Function to check for next waypoint with shortest distance"""
    previous_nodes, shortest_path = dijkstra_algorithm(graph=graph, start_node=start_node)
    
    valid_points = {k: v for k, v in shortest_path.items() if k in waypoints}
    min_distance = min([value for value in valid_points.values() if value != 0])
    next_stop = [key for key in valid_points if valid_points[key] == min_distance][0]
    waypoints.remove(next_stop)

    return next_stop, min_distance

total_distance = 0
all_stops = []

def find_waypoints_path(waypoints, start_node, target_node):
    """Run Dijkstra's algoritm recursively to look for shortest path to all waypoints,
    when all waypoints are visited, find shortest path to final node.
    """
    global total_distance
    global all_stops
    
    # If there are waypoints left and we are not at the target_node, run Dijkstra.
    # Store the closest waypoint and it's distance and rerun this function    
    if start_node != target_node and len(waypoints) > 0:
        if all_stops.count(start_node) == 0:
            all_stops.append(start_node)
        next_stop, min_distance = check_waypoints(waypoints, start_node, target_node)
        total_distance += min_distance
        all_stops.append(next_stop)
        find_waypoints_path(waypoints, next_stop, target_node)
        
    else:
        # if there are no waypoints left, find shortest path to the target_node
        previous_nodes, shortest_path = dijkstra_algorithm(graph=graph, start_node=start_node)
        
        temp_path = []
        node = target_node
        while node != start_node:
            temp_path.append(node)
            node = previous_nodes[node]
        temp_path.reverse()
        all_stops.extend(temp_path)
        final_distance = shortest_path[target_node]
        total_distance += final_distance
        
        # print results
        print(f"Het kortste pad is {total_distance} supermarkt-pad-lengte-eenheden lang")
        print(" -> ".join(all_stops))
        return all_stops

waypoints_path = find_waypoints_path(b_path, "ingang", "uitgang")
check_path(b_path, waypoints_path)

Het kortste pad is 48 supermarkt-pad-lengte-eenheden lang
ingang -> brood -> snacks -> koelschap -> bakproducten -> dranken -> kassa -> uitgang
Alle benodigde afdelingen zijn bezocht!
